In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic


In [ ]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [ ]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

In [ ]:
# how many recent turns you want to keep verbatim (e.g. last 3 turns)
summary_cutoff = 3

In [ ]:
# Let's make a conversation between GPT-4o-mini, Gemini and Claude-3-haiku, and gemini-2.0-flash
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model="gemini-2.5-flash-preview-04-17",

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gemini_system = "You are optimistic, funny chatbot. You try to make fun of everything the other person say, without ridicule, just rty to make the other have fun as well."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["What's up"]

In [ ]:
def summarize_conversation():
    summary_messages = []
    for gpt, claude, gemini in zip(
        gpt_messages[:-summary_cutoff],
        claude_messages[:-summary_cutoff],
        gemini_messages[:-summary_cutoff]
    ):
        summary_messages.append(f"GPT: {gpt}\nClaude: {claude}\nGemini: {gemini}")

    full_history = "\n---\n".join(summary_messages)

    summary_prompt = f"""
Summarize the following multi-agent conversation between GPT, Claude, and Gemini. 
Keep it brief, only include key ideas or themes.

Conversation:
{full_history}
"""

    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=[
            {"role": "system", "content": "You are a summarizer for multi-agent AI conversations."},
            {"role": "user", "content": summary_prompt}
        ],
        max_tokens=250
    )

    return completion.choices[0].message.content.strip()


In [ ]:
# unified plain-text streaming interface that works in any Python environment
def stream_model_response(name, stream, chunk_getter, display=True):
    if display:
        print(f"\n{name}:\n", end="", flush=True)
    full_response = ""
    if not display:
        yield f"{name}:\n"
    for chunk in stream:
        text = chunk_getter(chunk)
        if text:
            full_response += text
            if display:
                print(text, end="", flush=True)
            if not display:
                yield text
    if not display:           
        yield "\n"

    if display:
        print("\n")  # End with newline for clean formatting
    return full_response


In [ ]:
def call_claude(display=True):
    messages = []

    if len(gpt_messages) > summary_cutoff:
        summary = summarize_conversation()
        messages.append({"role": "user", "content": f"(Summary of previous conversation):\n{summary}"})

    for gpt, gemini, claude_message in zip(
        gpt_messages[-summary_cutoff:],
        gemini_messages[-summary_cutoff:],
        claude_messages[-summary_cutoff:]
    ):
        messages.append({"role": "user", "content": f"GPT: {gpt}\nGemini: {gemini}"})
        messages.append({"role": "assistant", "content": claude_message})

    # Keep conversation going with latest input
    messages.append({"role": "user", "content": f"GPT: {gpt_messages[-1]}\nGemini: {gemini_messages[-1]}"})

    stream = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500,
        stream=True  # Required for streaming
    )

    return stream_model_response("Claude", stream, lambda e: e.delta.text if e.type == "content_block_delta" else "", display)


In [ ]:
def call_gpt(display=True):
    messages = [{"role": "system", "content": gpt_system}]

    if len(gpt_messages) > summary_cutoff:
        summary = summarize_conversation()
        messages.append({"role": "user", "content": f"(Summary of previous conversation):\n{summary}"})

    for claude, gemini, gpt in zip(
        claude_messages[-summary_cutoff:],
        gemini_messages[-summary_cutoff:],
        gpt_messages[-summary_cutoff:]
    ):
        messages.append({"role": "user", "content": f"Claude: {claude}\nGemini: {gemini}"})
        messages.append({"role": "assistant", "content": gpt})

    messages.append({"role": "user", "content": f"Claude: {claude_messages[-1]}\nGemini: {gemini_messages[-1]}"})

    stream = openai.chat.completions.create(
        model=gpt_model,
        messages=messages,
        stream=True  # Enable streaming
    )

    return stream_model_response("GPT", stream, lambda c: c.choices[0].delta.content or "", display)


In [ ]:
def call_gemini(display=True):
    messages = []

    if len(gpt_messages) > summary_cutoff:
        summary = summarize_conversation()
        messages.append({
            "role": "user",
            "parts": [{"text": f"(Summary of earlier conversation):\n{summary}"}]
        })

    # Add last few verbatim turns
    for gpt, claude, gemini in zip(
        gpt_messages[-summary_cutoff:],
        claude_messages[-summary_cutoff:],
        gemini_messages[-summary_cutoff:]
    ):
        messages.append({
            "role": "user",
            "parts": [{"text": f"Claude: {claude}\nGPT: {gpt}"}]
        })
        messages.append({"role": "assistant", "parts": [{"text": gemini}]})

    # Add latest user turn
    messages.append({
        "role": "user",
        "parts": [{"text": f"Claude: {claude_messages[-1]}\nGPT: {gpt_messages[-1]}"}]
    })

    # gemini = google.generativeai.GenerativeModel(
    #     model_name='gemini-2.0-flash',
    #     system_instruction=gemini_system
    # )

    # completion = gemini.generate_content(messages)
    # return completion.text
    gemini = google.generativeai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=gemini_system
    )

    stream = gemini.generate_content(messages, stream=True)

    return stream_model_response("Gemini", stream, lambda c: c.text or "", display)



In [ ]:
# No interface
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["What's up"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    gemini_messages.append(gemini_next)

In [ ]:
import gradio as gr
import time

def run_conversation():
    gpt_messages = ["Hi there"]
    claude_messages = ["Hi"]
    gemini_messages = ["What's up"]

    full_output = f"GPT:\n{gpt_messages[0]}\n\nClaude:\n{claude_messages[0]}\n\nGemini:\n{gemini_messages[0]}\n\n"
    yield full_output

    for i in range(5):
        full_output += f"Turn {i + 1}:\n"

        # GPT stream
        gpt_stream = call_gpt(display=False)
        gpt_next = ""
        for chunk in gpt_stream:
            gpt_next += chunk
            partial_turn_output = full_output + f"GPT: {gpt_next}\n"
            yield partial_turn_output
        gpt_messages.append(gpt_next)
        full_output += f"{gpt_next}\n"

        # Claude stream
        claude_stream = call_claude(display=False)
        claude_next = ""
        for chunk in claude_stream:
            claude_next += chunk
            partial_turn_output = full_output + f"Claude: {claude_next}\n"
            yield partial_turn_output
        claude_messages.append(claude_next)
        full_output += f"{claude_next}\n"

        # Gemini stream
        gemini_stream = call_gemini(display=False)
        gemini_next = ""
        for chunk in gemini_stream:
            gemini_next += chunk
            partial_turn_output = full_output + f"Gemini: {gemini_next}\n"
            yield partial_turn_output
        gemini_messages.append(gemini_next)
        full_output += f"{gemini_next}\n\n"
        
        time.sleep(1)  # Optional, simulate delay
    yield full_output

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Multi-Model Conversation Viewer")
    start_button = gr.Button("Start Conversation")
    chat_output = gr.Textbox(label="Conversation", lines=30)

    start_button.click(fn=run_conversation, outputs=chat_output)

demo.launch()
